In [1]:
from os import path
import numpy as np
import pandas as pd
import natsort
import glob
import cv2
import os
import tifffile as tiff
import imageio
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage import exposure
import scipy.io
import anndata
from skimage.transform import warp
from skimage.draw import polygon
from PIL import Image
import seaborn as sns
import dask.array as da
import imagecodecs
import math
import skimage
from skimage import transform
import scanpy as sc
from scipy.sparse import issparse
from scipy.stats import linregress

# Load Xenium Result

In [4]:
adata_xenium = sc.read_10x_h5(
    filename="path/to/file/cell_feature_matrix.h5"
)
stitched_image = np.load('stitched_images_BFP/stitched_BFP.npy')

In [5]:
csv_file_path = 'path/to/file/cell_boundaries.csv.gz'
cell_boundaries_csv = pd.read_csv(csv_file_path)

cell_boundaries_csv['vertex_x_coord'] = cell_boundaries_csv['vertex_x']/0.2125
cell_boundaries_csv['vertex_y_coord'] = cell_boundaries_csv['vertex_y']/0.2125

affine_matrix = np.loadtxt('path/to/file/matrix.csv', delimiter=',')
affine_matrix_inv = np.linalg.inv(affine_matrix)

homogeneous_coordinates = np.hstack([cell_boundaries_csv[['vertex_x_coord', 'vertex_y_coord']].values, 
                                     np.ones((cell_boundaries_csv.shape[0], 1))])
transformed_coordinates = homogeneous_coordinates.dot(affine_matrix_inv.T)
cell_boundaries_csv['x_transformed'] = transformed_coordinates[:, 0]
cell_boundaries_csv['y_transformed'] = transformed_coordinates[:, 1]

cell_boundaries_csv['x_transformed'] = round(cell_boundaries_csv['x_transformed']).astype(int)
cell_boundaries_csv['y_transformed'] = round(cell_boundaries_csv['y_transformed']).astype(int)


In [7]:
cell_boundaries_csv_agg = cell_boundaries_csv.groupby('cell_id').agg({
    'x_transformed': ['min', 'max'],
    'y_transformed': ['min', 'max']
})
cell_boundaries_csv_agg.columns = ['_'.join(c) for c in cell_boundaries_csv_agg.columns]

In [9]:
valid_ids = cell_boundaries_csv_agg[
    (cell_boundaries_csv_agg['x_transformed_min'] >= 0) &
    (cell_boundaries_csv_agg['x_transformed_max'] < stitched_image.shape[1]) &
    (cell_boundaries_csv_agg['y_transformed_min'] >= 0) &
    (cell_boundaries_csv_agg['y_transformed_max'] < stitched_image.shape[0])
].index

cell_boundaries_csv = cell_boundaries_csv[cell_boundaries_csv['cell_id'].isin(valid_ids)]

# Run for all FOVs 

In [36]:
for FOV_num in range(1, 465):
    if FOV_num % 15 == 0:
        continue
    num_of_row = FOV_num // 15 + 1
    num_of_col = FOV_num % 15
    y_sub = int((num_of_row - 1) * 2000 * 0.98)
    x_sub = int((num_of_col - 1) * 2000 * 0.98)

    df_fov = cell_boundaries_csv.copy()
    df_fov['x_transformed_subed'] = df_fov['x_transformed'] - x_sub
    df_fov['y_transformed_subed'] = df_fov['y_transformed'] - y_sub
    valid_ids = cell_boundaries_csv_agg[
        (cell_boundaries_csv_agg['x_transformed_min'] >= x_sub) &
        (cell_boundaries_csv_agg['x_transformed_max'] < x_sub + 2000) &
        (cell_boundaries_csv_agg['y_transformed_min'] >= y_sub) &
        (cell_boundaries_csv_agg['y_transformed_max'] < y_sub + 2000)
    ].index
    df_fov = df_fov[df_fov['cell_id'].isin(valid_ids)]
    
    field_str = f"{FOV_num:04d}"
    base = f"Field{field_str}_stack"
    results_dir = "TrackMateResults"
    spots_path  = f"{results_dir}/Field{field_str}/{base}-spots.csv"
    tracks_path = f"{results_dir}/Field{field_str}/{base}-tracks.csv"
    spots  = pd.read_csv(spots_path,  header=0, skiprows=[1,2,3], index_col=0)
    tracks = pd.read_csv(tracks_path, header=0, skiprows=[1,2,3], index_col=0)
    
    spots['POSITION_X_p'] = (spots['POSITION_X'] * 100).clip(upper=1999)
    spots['POSITION_Y_p'] = (spots['POSITION_Y'] * 100).clip(upper=1999)
    spots = spots.assign(
        col=spots['POSITION_X_p'].round().astype(int),
        row=spots['POSITION_Y_p'].round().astype(int),
    )
    
    height, width = (2000, 2000)
    df_list = []
    for cell_id, grp in df_fov.groupby('cell_id'):
        xs = grp['x_transformed_subed'].to_numpy()
        ys = grp['y_transformed_subed'].to_numpy()
        cols = np.round(xs).astype(int)
        rows = np.round(ys).astype(int)
        rr, cc = polygon(rows, cols, shape=(height, width))
        df_cell = pd.DataFrame({
            'cell_id': cell_id,
            'row': rr,
            'col': cc,
        })
        df_list.append(df_cell)
    pixels_df = pd.concat(df_list, ignore_index=True)
    
    merged = spots.merge(pixels_df, on=['row','col'], how='left')
    total_pos = spots.groupby('TRACK_ID').size().rename('total_positions')
    in_cell = (
        merged
        .dropna(subset=['cell_id'])
        .groupby(['TRACK_ID','cell_id'])
        .size()
        .rename('positions_in_cell')
    )
    result = (
        in_cell
        .reset_index()
        .merge(total_pos.reset_index(), on='TRACK_ID')
    )
    result = result[
        (result['positions_in_cell'] > 0) &
        (result['positions_in_cell'] < result['total_positions'])
    ].reset_index(drop=True)
    
    merged_result = result.merge(tracks, how='left', on='TRACK_ID')
    merged_result = merged_result[merged_result['TRACK_DISPLACEMENT'] > 0.05]
    merged_result['FOV'] = FOV_num
    
    out_path = (
        "Track_Cell_Result"
        f"/Field{field_str}_track_cell.csv"
    )
    merged_result.to_csv(out_path, index=False)
    print(f"Finished FOV {FOV_num} → wrote {out_path}")

Finished FOV 1 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0001_track_cell.csv
Finished FOV 2 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0002_track_cell.csv
Finished FOV 3 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0003_track_cell.csv
Finished FOV 4 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0004_track_cell.csv
Finished FOV 5 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0005_track_cell.csv
Finished FOV 6 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0006_track_cell.csv
Finished FOV 7 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0007_track_cell.csv
Finished FOV 8 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0008_track_c

Finished FOV 70 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0070_track_cell.csv
Finished FOV 71 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0071_track_cell.csv
Finished FOV 72 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0072_track_cell.csv
Finished FOV 73 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0073_track_cell.csv
Finished FOV 74 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0074_track_cell.csv
Finished FOV 76 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0076_track_cell.csv
Finished FOV 77 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0077_track_cell.csv
Finished FOV 78 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0078

Finished FOV 140 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0140_track_cell.csv
Finished FOV 141 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0141_track_cell.csv
Finished FOV 142 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0142_track_cell.csv
Finished FOV 143 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0143_track_cell.csv
Finished FOV 144 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0144_track_cell.csv
Finished FOV 145 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0145_track_cell.csv
Finished FOV 146 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0146_track_cell.csv
Finished FOV 147 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/F

Finished FOV 209 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0209_track_cell.csv
Finished FOV 211 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0211_track_cell.csv
Finished FOV 212 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0212_track_cell.csv
Finished FOV 213 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0213_track_cell.csv
Finished FOV 214 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0214_track_cell.csv
Finished FOV 215 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0215_track_cell.csv
Finished FOV 216 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0216_track_cell.csv
Finished FOV 217 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/F

Finished FOV 279 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0279_track_cell.csv
Finished FOV 280 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0280_track_cell.csv
Finished FOV 281 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0281_track_cell.csv
Finished FOV 282 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0282_track_cell.csv
Finished FOV 283 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0283_track_cell.csv
Finished FOV 284 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0284_track_cell.csv
Finished FOV 286 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0286_track_cell.csv
Finished FOV 287 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/F

Finished FOV 349 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0349_track_cell.csv
Finished FOV 350 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0350_track_cell.csv
Finished FOV 351 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0351_track_cell.csv
Finished FOV 352 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0352_track_cell.csv
Finished FOV 353 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0353_track_cell.csv
Finished FOV 354 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0354_track_cell.csv
Finished FOV 355 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0355_track_cell.csv
Finished FOV 356 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/F

Finished FOV 418 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0418_track_cell.csv
Finished FOV 419 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0419_track_cell.csv
Finished FOV 421 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0421_track_cell.csv
Finished FOV 422 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0422_track_cell.csv
Finished FOV 423 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0423_track_cell.csv
Finished FOV 424 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0424_track_cell.csv
Finished FOV 425 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/Field0425_track_cell.csv
Finished FOV 426 → wrote /scratch/welchjd_root/welchjd2/shared_data/Xenium_0421_Lyso/Track_Cell_Result/F

# Track Cell Analysis (Find the perturbed gene for the 4 example cells)

In [12]:
folder = "Track_Cell_Result"
all_files = glob.glob(os.path.join(folder, "Field*_track_cell.csv"))
all_files.sort()

df_list = []
for f in all_files:
    df = pd.read_csv(f)
    df_list.append(df)
combined = pd.concat(df_list, ignore_index=True)
combined['positions_out_cell'] = combined['total_positions'] - combined['positions_in_cell']
combined = combined[
    (combined['positions_in_cell'] > 2) &
    (combined['positions_out_cell'] > 2) &
    (combined['total_positions'] > 5)
]

/tmp/ipykernel_1455099/3009959699.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = pd.concat(df_list, ignore_index=True)


In [31]:
combined

,TRACK_ID,cell_id,positions_in_cell,total_positions,TRACK_INDEX,NUMBER_SPOTS,NUMBER_GAPS,NUMBER_SPLITS,NUMBER_MERGES,NUMBER_COMPLEX,...,TRACK_STD_SPEED,TRACK_MEAN_QUALITY,TOTAL_DISTANCE_TRAVELED,MAX_DISTANCE_TRAVELED,CONFINEMENT_RATIO,MEAN_STRAIGHT_LINE_SPEED,LINEARITY_OF_FORWARD_PROGRESSION,MEAN_DIRECTIONAL_CHANGE_RATE,FOV,positions_out_cell
1,39,bmeaflam-1,9,21,39,21,0,0,0,0,...,0.034830,139.588960,0.390646,0.157730,0.350778,0.006851,0.350778,1.290110,1,12
3,43,bmehfmbl-1,8,50,43,50,0,0,0,0,...,0.008074,139.385289,0.380153,0.183330,0.455376,0.003533,0.455376,1.046658,1,42
4,43,bmejpdaj-1,42,50,43,50,0,0,0,0,...,0.008074,139.385289,0.380153,0.183330,0.455376,0.003533,0.455376,1.046658,1,8
6,51,bmejpdaj-1,21,40,51,40,0,0,0,0,...,0.029394,378.429841,0.683258,0.304597,0.445801,0.007810,0.445801,1.858211,1,19
7,272,bmejpdaj-1,5,15,272,15,0,0,0,0,...,0.026370,197.360077,0.227862,0.171209,0.751369,0.012229,0.751369,1.561646,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261648,4361,gocmnlci-1,9,14,4361,14,0,0,0,0,...,0.031759,52.584036,0.205440,0.129442,0.562164,0.008884,0.562164,1.762969,464,5
261653,4453,ackoiddm-1,5,10,4453,10,1,0,0,0,...,0.019346,6.887331,0.283934,0.088574,0.230739,0.006551,0.222528,1.402091,464,5
261654,4453,gmpmgchh-1,5,10,4453,10,1,0,0,0,...,0.019346,6.887331,0.283934,0.088574,0.230739,0.006551,0.222528,1.402091,464,5
261655,4457,gmpkndjl-1,8,11,4457,11,1,0,0,0,...,0.043206,9.640182,0.351290,0.219336,0.624372,0.019940,0.571013,1.486425,464,3


In [33]:
#combined.to_csv("combined_tracks_and_cells.csv",index=False)

In [13]:

adata_xenium.obs['total_counts'] = (
        adata_xenium.X.sum(axis=1).A1
        if hasattr(adata_xenium.X, "A1")
        else adata_xenium.X.sum(axis=1)
    )
adata_xenium.obs['total_counts'] = adata_xenium.X.sum(axis=1).A1
num_cells_800 = (adata_xenium.obs['total_counts'] >= 800).sum()
adata_filtered = adata_xenium[adata_xenium.obs['total_counts'] >= 800].copy()
sgRNA_genes = [gene for gene in adata_filtered.var_names if '-' in gene and not 'HLA' in gene]
sgRNA_to_target = {sg: sg.split('-')[0] for sg in sgRNA_genes}

idxs = [adata_filtered.var_names.get_loc(g) for g in sgRNA_genes]
mat = adata_filtered.X[:, idxs]
if issparse(mat):
    mat = mat.toarray()
bin_mat = (mat > 0).astype(int)
df_ind = pd.DataFrame(
    bin_mat,
    index=adata_filtered.obs_names,
    columns=[g for g in sgRNA_genes]
)
adata_filtered.obs = adata_filtered.obs.join(df_ind)

obs_clean = adata_filtered.obs.drop(columns=["total_counts"], errors="ignore")
gene_cols = [g for g in sgRNA_genes if g in obs_clean.columns]

counts = obs_clean[gene_cols].sum(axis=0).sort_values(ascending=False)
counts_df = pd.DataFrame({
    "gene":  counts.index,
    "count": counts.values
})
counts_df["rank"] = counts_df["count"].rank(ascending=False, method="dense")

top4 = counts_df.loc[:3, 'gene'].tolist()
remaining = [
    col for col in adata_filtered.obs.columns
    if col not in top4 + ['total_counts']
]
adata_filtered.obs['sgRNA_Gene_count'] = (
    adata_filtered.obs[remaining].sum(axis=1)
)
adata_filtered.obs['perturbation'] = np.where(
    adata_filtered.obs['sgRNA_Gene_count'] > 0,
    'perturbed',
    'unperturbed'
)
cell_perturb_ind = adata_filtered.obs.copy()[['sgRNA_Gene_count', 'perturbation']]

In [14]:
top4

['ID2-4', 'SOX5-3', 'NFIB-2', 'ID2-2']

In [15]:
obs = adata_filtered.obs[remaining]  # a pandas.DataFrame, indexed by cell_id
sgRNA_cols = [col for col in obs.columns if "-" in col]

gene_names = sorted({col.split("-", 1)[0] for col in sgRNA_cols})
perturb_df = pd.DataFrame(index=obs.index, columns=gene_names, dtype="object")

for gene in gene_names:
    matching_cols = [col for col in sgRNA_cols if col.startswith(gene + "-")]
    if not matching_cols:
        continue

    sub_df = obs[matching_cols]
    any_positive = (sub_df > 0).any(axis=1)
    perturb_df[gene] = any_positive.map({True: "perturbed", False: "unperturbed"})
print("First few rows of perturb_df:")
print(perturb_df.iloc[:5, :5])  # show a small 5×5 corner
for gene in gene_names:
    colname = f"{gene}_perturb_status"
    adata_filtered.obs[colname] = perturb_df[gene]


First few rows of perturb_df:
                   AFF1         BCL6         BNC2      CARHSP1        CREB5
aaadneob-1  unperturbed  unperturbed  unperturbed  unperturbed  unperturbed
aaahmode-1    perturbed  unperturbed  unperturbed  unperturbed  unperturbed
aaahnoof-1  unperturbed  unperturbed  unperturbed  unperturbed  unperturbed
aaajgnge-1  unperturbed  unperturbed  unperturbed  unperturbed  unperturbed
aaaohebe-1  unperturbed  unperturbed  unperturbed  unperturbed  unperturbed


In [16]:
cell_list = ['boljjmgm-1', 'bolicola-1', 'chhohmfi-1', 'bolonabg-1',]

In [18]:
subset = perturb_df.loc[cell_list]

perturbed_per_cell = subset.apply(
    lambda row: row[row == "perturbed"].index.tolist(),
    axis=1
)

genes_perturbed_any = subset.columns[(subset == "perturbed").any()].tolist()
genes_perturbed_all = subset.columns[(subset == "perturbed").all()].tolist()

print("Perturbed genes per cell:")
print(perturbed_per_cell)

print("\nGenes perturbed in any of the four cells:")
print(genes_perturbed_any)

print("\nGenes perturbed in all four cells:")
print(genes_perturbed_all)


Perturbed genes per cell:
boljjmgm-1            [HMGB2]
bolicola-1          [ONECUT2]
chhohmfi-1    [ISL1, ONECUT1]
bolonabg-1             [ISL1]
dtype: object

Genes perturbed in any of the four cells:
['HMGB2', 'ISL1', 'ONECUT1', 'ONECUT2']

Genes perturbed in all four cells:
[]
